# Main programme avec interaction avec l'utilisateur



L'idée de ce dossier est de modéliser un CP avec interaction d'un décision-maker.

In [4]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from docplex.cp.config import get_default
import numpy as np
from Solver import *
from User import *
import time

In [5]:
#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# Interaction with the solver

data = []

with open('./example.data', 'r') as f:    
    for line in f:
        data.append(line.strip().split())

n = int(data[0][0])
m = int(data[0][1])

T_machine = []
T_duration = []

for i in range(1,n+1):
    for j in range(0, 2*m, 2):
        T_machine.append(int(data[i][j]))
        T_duration.append(int(data[i][j+1]))

duration = np.zeros((n, m))

for i in range(n):
    for j in range(m):
        ind_machine = int(T_machine[i*m + j])
        duration[i][ind_machine] = T_duration[i*m + j]


# --------- Call Solver constructor in Solver.py

solver = Solver(data)

model = CpoModel() 

# --------- Create the model variables 

solver.create_variables(model, n, m, duration)


# ------------ Add objective function to the solver

# print("\nAdding objective function to the solver...")
# # print(variables[i][T_machine[i*m + m -1]])
# makespan = max([end_of(variables[i][T_machine[i*m + m -1]]) for i in range(n)])
# solver.add_constraint(model.minimize(makespan))
# print("Objective function added !")

# ------------ Solve the model

print("\nSolving the model...")
msol, nb_solution, runtime = solver.solve(model, 60, n, m, T_machine)

# ------------ Display the result

for sol in msol:
    sol.write()
print("Model solved !")


# ---------------- Interaction with the user

print("\n--------Interaction with the user...---------")

print("\nCreating the user...")
user = User()
print("User created !")

print("\nClassing solutions...")	
list_indice, list_equal = user.classerSolutions(msol)
print("Solutions classed !")

print("\nCreating preferences...")
pref = user.getPreferences()
print("Preferences created !")

print("\nDisplaying preferences...")
# for sol in pref:
#     sol.write()

print("\nPreferences : ", pref)
print(list_indice)
print(list_equal)



--------Main program is loading...---------

Solver initialized !

Creating the model variables...
Model variables created !

Solving the model...

Adding precedence constraints to the solver...
Precedence constraints added !

Adding disjunctive constraints to the solver...
Disjunctive constraints added !
-------------------------------------------------------------------------------
Model constraints: 36, variables: integer: 0, interval: 36, sequence: 6
Solve status: Feasible
Search status: SearchOngoing, stop cause: SearchHasNotBeenStopped
Solve time: 0.07 sec
-------------------------------------------------------------------------------
Variables:
   T0-0 = IntervalVarValue(start=6, end=9, size=3)
   T0-1 = IntervalVarValue(start=16, end=22, size=6)
   T0-2 = IntervalVarValue(start=5, end=6, size=1)
   T0-3 = IntervalVarValue(start=22, end=29, size=7)
   T0-4 = IntervalVarValue(start=54, end=60, size=6)
   T0-5 = IntervalVarValue(start=29, end=32, size=3)
   T1-0 = IntervalVarValu

In [6]:
###  -------------- Iteration of the solver with the preferences

# ----------------- Add the preferences to the model

tps_max = 100
tps = runtime
it_max = 5
it = 1
k = 1

criterion = (tps < tps_max) and (it < it_max) 

while criterion :

    it += 1
    
    solver = Solver(data)
    model = CpoModel() 

    # --------- Create the model variables 

    solver.create_variables(model, n, m, duration) 

    # stop = int(input("Do you want to continue ? True(1) or False(0)"))

    bb = integer_var(0,1)
    solver.add_variable(bb)
    bb=1

    list_variables = solver.get_variables()
    variables = list_variables[0]
    print(variables)

    for sol in pref:
        b = integer_var(0,1)
        solver.add_variable(b)
        b=0
        for i in range(n):
            for j in range(m):

                var_sol = sol.get_all_var_solutions()[i*m + j]
                #a = model.interval_var(start = sol[variables[i][j]].start, end= sol[variables[i][j]].end, size=int(duration[i][j]), name="a{}{}".format(i,j))
                #b =max(b,logical_or((model.start_of(a) != model.start_of(variables[i][j])), (model.end_of(a) != model.end_of(variables[i][j]))))
                
                b =max(b,logical_or(var_sol.get_start() != model.start_of(variables[i][j]), var_sol.get_end() != model.end_of(variables[i][j])))
        b = (b!=0)
        bb = bb * b
    solver.add_constraint(bb==1)
    

# ------------ Solve the model

    print("\nSolving the model...")

    msol, nb_solution, runtime = solver.solve(model, k, n, m, T_machine)

# ------------ Display the result

    for sol in msol:
        sol.write()
    print("Model solved !")


# ---------------- Interaction with the user

    print("\n--------Interaction with the user...---------")

    print("\nClassing solutions...")	
    list_indice, list_equal = user.classerSolutions(msol)
    print("Solutions classed !")

    print("\nCreating preferences...")
    pref = user.getPreferences()
    print("Preferences created !")

    print("\nDisplaying preferences...")
    # for sol in pref:
    #     sol.write()

    print("\nPreferences : ", pref)
    print(list_indice)
    print(list_equal)
    print(len(pref))

    tps += runtime

    criterion = (tps < tps_max) and (it < it_max)

    if (it == it_max):
        print("The user has reached the maximum number of iterations !")
    # elif not stop:
    #     print("The user has chosen his/her solution !")
    elif (tps >= tps_max):
        print("The user has reached the maximum time !")


print("\n--------End of the interaction with the user...---------")


Solver initialized !

Creating the model variables...
Model variables created !
[[<docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F45F0>, <docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F4660>, <docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F47B0>, <docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F4AC0>, <docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F4B30>, <docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F4BA0>], [<docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F4C10>, <docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F4C80>, <docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F4CF0>, <docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F4D60>, <docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F4DD0>, <docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F4E40>], [<docplex.cp.expression.CpoIntervalVar object at 0x0000027E550F4EB0>, <docplex.cp.expr